In [1]:
from pykafka import KafkaClient
from pykafka.common import OffsetType
import time


In [2]:
client = KafkaClient("127.0.0.1:9092")

In [10]:
def get_messages(topicname):
    consumer = client.topics[topicname].get_simple_consumer(auto_offset_reset=OffsetType.LATEST, reset_offset_on_start=True)
    offsets = [(p, op.last_offset_consumed)
           for p, op in consumer._partitions.items()]
    offsets = [(p, (o if o > -1 else -2)) for p, o in offsets]
    consumer.reset_offsets(offsets)
    def events():
        for message in consumer:
            msg = message.value.decode()
            yield f"{time.time()}|topic:{topicname}|message:{msg}"
            if msg=='stop':
                print('stopping')
                break
    return events()

In [11]:
for x in get_messages("test"):
    print(x)

1592439187.8582103|topic:test|message:this is a test
1592439192.311003|topic:test|message:12341234
1592439195.7307491|topic:test|message:asdf
1592439209.1371086|topic:test|message:stop
stopping
